In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from model import ResidualBlock, Discriminator
import torchvision
import torch

image = torchvision.transforms.Resize(256)(torchvision.io.read_image("face.jpg").float()).unsqueeze(0)

In [ ]:
image.size()

In [ ]:
rb = ResidualBlock(3, 16)

In [ ]:
out = rb(image)
out.size()

In [ ]:
d = Discriminator(7, 16)

In [ ]:
print(image.size())
d(image)

In [ ]:
test = torch.zeros((8,3,256,256))
d(test).size()

In [ ]:
import torch
torch.manual_seed(9)
#layer = torch.nn.Linear(1, 4)
# layer.weight *= 4 inplace operations on leaf variables does not work

con_layer = torch.nn.Conv2d(4,16, kernel_size=3)
test = torch.zeros((8,4,256,256))
test_unf = torch.nn.functional.unfold(test, (3, 3))
weight = torch.randn(size=(16, 4, 3, 3))
a = test_unf.transpose(1, 2).matmul(weight.view(weight.size(0), -1).t())
print(a.size())
print(con_layer.weight.size(), con_layer(test).size(), test.size(), test_unf.size())

In [ ]:
import torch
from torch import nn
unfold = nn.Unfold(kernel_size=(2, 3))
input = torch.randn(2, 5, 3, 4)
output = unfold(input)
# each patch contains 30 values (2x3=6 vectors, each of 5 channels)
# 4 blocks (2x3 kernels) in total in the 3x4 input
# print(input.size(),output.size())

# Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)
inp = torch.randn(4, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
print(inp_unf.size())
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
print(out_unf.size())
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))
print(out.size())
# or equivalently (and avoiding a copy),
# out = out_unf.view(1, 2, 7, 8)
(torch.nn.functional.conv2d(inp, w) - out).abs().max()

In [ ]:
w.view(w.size(0), -1).t().size()

In [ ]:
torch.manual_seed(9)
def convolution(i, w):

    inp_unf = torch.nn.functional.unfold(i, (w.size(2), w.size(3)))
    if len(w.size()) == 4:
        out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
    else:
        print("heuy", inp_unf.transpose(1, 2).size(), w.view(w.size(0), w.size(1), -1).transpose(1,2).size())
        out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), w.size(1), -1).transpose(1,2)).transpose(1, 2)
    out = torch.nn.functional.fold(out_unf, (i.size(2)-w.size(2)+1, i.size(3)-w.size(3)+1), (1, 1))
    print(out.size())
    return out

In [ ]:
torch.manual_seed(9)
i = torch.ones((8,3,6,6))
s2 = torch.rand(8,1,3,1,1)
s1 = s2.view(8,3,1,1)
# print(s)
w = torch.ones((16, 3, 3, 3))
o1 = convolution(i*s1, w)
bias = torch.randn((16,1,1))
print(o1+bias)
print((s2*w).size())
o2 = convolution(i, s2*w)
print((~(torch.abs(o1-o2) <1e-3)).sum())

In [ ]:
a = torch.randn((8,3,2,1,1))
a.square().sum(dim=(2,3,4), keepdim=True).size()

In [ ]:
a,a.square().sum(dim=(2,3,4))

In [ ]:
from model import ModDemodConv3x3
mdconv = ModDemodConv3x3(3, 16)
i = torch.ones((8,3,256,256))
s = torch.rand(8,3)
o=mdconv(i,s)
o.size()

In [ ]:
input = torch.arange(1, 5, dtype=torch.float32).view(1, 1, 2, 2)
print(input.size())

m = torch.nn.Upsample(scale_factor=2, mode='nearest')
print(m(input).size())

m = torch.nn.Upsample(scale_factor=2, mode='bilinear')  
print(m(input).size())

In [ ]:
x = torch.tensor([[1], [2], [3]])
print(x.size())
print(x.repeat(1,4).size())   # -1 means not changing the size of that dimension
print(x, x.repeat(1,4))

In [ ]:
x.unsqueeze(0).size()

In [ ]:
torch.zeros([]).size()

In [ ]:
noise_scaling_factor = torch.tensor([2])
noise = torch.randn((1,1,4,4))

In [ ]:
noise, noise_scaling_factor * noise, noise_scaling_factor.size()

In [ ]:
torch.zeros((1))*noise

In [ ]:
upsample = torch.nn.Upsample(scale_factor=2, mode="bilinear")
inp = torch.randn(8,3,4,4)
inp.size(), upsample(inp).size()

In [ ]:
layer = torch.nn.Conv2d(3,16,3)
layer.weight.size(), layer.bias.size()

In [ ]:
inp = torch.randn((8,3,16,16))

In [ ]:
layer(inp).size()

In [ ]:
import torch
from model import HistoGAN
from torchinfo import summary

In [ ]:
from data import AnimeFacesDataset
from torchvision.transforms import Resize
resize = Resize((256,256))
image_dir = "images"
dataset = AnimeFacesDataset(image_dir)

In [ ]:
dataset[0].size()